# Modules, path, privacy

## modules

- 所有的rust项目都会组织成一棵模块树，根是crate
- 模块只有两个功能，一是组织代码，而是定义可见性
- path就是模块在树中的路径，也是引用模块的方法
- pub关键字让模块内容在外部可见

In [5]:
mod front_of_house {
    pub mod hosting {
        pub fn add_to_wait_list(){
            println!("in wait list adding.");
        }
        fn sear_at_table(){}
    }
    mod serving {
        fn take_order(){}
        fn serve_order(){}
        fn take_payment(){}
    }
}

In [8]:
// 使用绝对路径访问函数
crate::front_of_house::hosting::add_to_wait_list();

in wait list adding.


In [10]:
// 使用相对路径访问函数
front_of_house::hosting::add_to_wait_list();

in wait list adding.


In [11]:
// 定义一个函数使用add_to_wait_list
fn have_dinner() {
    use front_of_house::hosting;
    hosting::add_to_wait_list();
}

In [12]:
// have_dinner属于模块crate
crate::have_dinner();

in wait list adding.


In [13]:
mod dinning {
    pub fn have_dinner(){ // 在不同模块中可以定义同名的函数
        use super::front_of_house::hosting; // 使用super引用父级模块内容
        hosting::add_to_wait_list();
    }
}

In [14]:
dinning::have_dinner();

in wait list adding.


### 模块中struct和enum的可见性

- module中的函数，struct和enum默认private，要公开可见，必须使用pub
- 公有struct中的字段默认private，要公开可见，必须对每个公有字段使用pub
- 公有enum中的字段默认public

In [16]:
mod back_of_house {
    #[derive(Debug)]
    pub enum MainCourse { // enum为pub，所有字段默认为pub
        Bread,
        Rice,
    }
    #[derive(Debug)]
    pub struct Dinner { // Dinner为pub，mc为pub，fruid为private
        pub mc: MainCourse,
        fruid: String,
    }
    impl Dinner {
        pub fn new_dinner(course: MainCourse) -> Dinner { 
            // 由于fruid为private，因此必须有这个函数用于创建Dinner struct
            Dinner {
                mc: course,
                fruid: String::from("Banana"),
            }
        }
    }
}

In [18]:
back_of_house::Dinner::new_dinner(back_of_house::MainCourse::Bread)

Dinner { mc: Bread, fruid: "Banana" }

### use技巧

- 当引入函数时，请保留函数的父级模块，然后使用`module::func()`的语法来调用
- 当引入struct，enum时，不要保留函数的父级模块，直接引入到具体的结构体或枚举
- 上一条规则有个例外，如果你把两个同名的结构体或枚举引入到统一模块当中，你需要保留父级模块来指定，或者采用`as`别名的方式

因此上一行代码应该改写为：

In [19]:
use back_of_house::{Dinner, MainCourse};
Dinner::new_dinner(MainCourse::Rice)

Dinner { mc: Rice, fruid: "Banana" }

### pub use

当编写lib代码时，我们可以通过pub use将内层模块的函数，结构体或枚举重新公有输出给外部调用者。例如我们想将MainCourse输出给外部调用者直接调用，可以写：

```rust
pub use back_of_house::MainCourse;
```

假设这个lib打包成restaurant，那么外部用户就可以使用

```rust
extern crate restaurant;

use restaurant::MainCourse;
```

来引入MainCourse了。

### 引入外部包

在`cargo`的`cargo.toml`文件中加入依赖，cargo会自动帮你下载外部的包及相应依赖

```toml
[dependencies]
rand = "0.5.5"
```

与下面notebook的指令一致：

In [23]:
:dep rand="0.5.5"

In [24]:
use rand::Rng;
rand::thread_rng().gen_range(1, 1001)

704